# Projeto Final | Analytics Engineering
----
**Engenharia de Dados e Garantia de Qualidade no Conjunto de Dados do Airbnb no Rio de Janeiro**

# Etapa 1:

**Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**

   - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
   - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

## Bibliotecas

In [182]:
from sqlalchemy import create_engine, text as sql_text
import pandas as pd
import datetime

import os

### Chamada da biblioteca para usar o SQL

In [2]:
%load_ext sql

## Informações do banco de dados

In [3]:
## Importar modulo de conexão e implementação do banco  
import modules.database as db

Estabelecer uma conexão com um banco de dados PostgreSQL

In [4]:
# Configurar conexão
engine = db.engine_db()

In [5]:
# Testar a conexão
try:
    with engine.connect():
        print("Conexão bem sucedida!")
except Exception as e:
    print("Erro ao conectar:", e)

Conexão bem sucedida!


## Camada Landing - Extração de dados

In [7]:
## Importar modulo para extração dos dados
from modules.etl import Landing

url = [
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/listings.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/calendar.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/reviews.csv.gz'
]
data_path = './data/'
landing_step = Landing(landing_path=data_path)
landing_step.extract(url=url)
landing_step.transform()

O diretório ./data/ não existe e tentaremos criar.
Diretório criado com sucesso!!
Efetuando a requisição para a url https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/listings.csv.gz
Efetuando a requisição para a url https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/calendar.csv.gz
Efetuando a requisição para a url https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/reviews.csv.gz
Salvando o arquivo ./data/calendar.csv.gzSalvando o arquivo ./data/reviews.csv.gz
Salvando o arquivo ./data/listings.csv.gz



## Preparação do Datalake

In [8]:
db.criar_database()
db.criar_schemas()

O banco de dados airbnb_RJ já existe.
O Schema 'bronze' ja existe.
O Schema 'silver' ja existe.
O Schema 'gold' ja existe.


## Camada Bronze - Ingestão e Análise dos Dados

In [9]:
try:
    print("Iniciando a estapa bronze.")

    ## Listings
    data_path = './data/'
    df_listings = pd.read_csv(data_path + 'listings.csv') # Carregando o csv listings em um Dataframe Python
    db.criar_tabela_df('bronze', 'listings', df_listings,'replace') # Criar e salvar o DataFrame na tabela "listings" dentro do esquema "bronze"

    ## Reviews
    df_reviews = pd.read_csv(data_path + 'reviews.csv') # Carregando o csv reviews em um Dataframe Python
    db.criar_tabela_df('bronze', 'reviews', df_reviews,'replace') # Criar e salvar o DataFrame na tabela "reviews" dentro do esquema "bronze"

    ## Calendar
    df_calendar = pd.read_csv(data_path + 'calendar.csv') # Carregando o csv calendar em um Dataframe Python
    db.criar_tabela_df('bronze', 'calendar', df_calendar,'replace') # Criar e salvar o DataFrame na tabela "calendar" dentro do esquema "bronze"

    print("Etapas bronze executada com sucesso!!")

except Exception as e:
    print("Ocorreu um erro inesperado ao efetua a etapa bronze:", e)

Iniciando a estapa bronze.
Criando a tabela 'listings' no schema 'bronze'....
A tabela 'listings' foi criada no schema 'bronze', e os dados foram inseridos.
Criando a tabela 'reviews' no schema 'bronze'....
A tabela 'reviews' foi criada no schema 'bronze', e os dados foram inseridos.
Criando a tabela 'calendar' no schema 'bronze'....
A tabela 'calendar' foi criada no schema 'bronze', e os dados foram inseridos.
Etapas bronze executada com sucesso!!


# Etapa 2:

**Data Clean - Camada Silver**

   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

## Camada Silver - Limpeza de Dados e Criação de colunas

### Leitura das tabelas bronze

In [181]:
query = """
SELECT * FROM bronze.listings
"""
df_silver_listings = pd.read_sql(sql=sql_text(query), con=engine.connect())

In [11]:
query = """
SELECT * FROM bronze.reviews
"""
df_silver_reviews = pd.read_sql(sql=sql_text(query), con=engine.connect())

In [12]:
query = """
SELECT * FROM bronze.calendar
"""
df_silver_calendar = pd.read_sql(sql=sql_text(query), con=engine.connect())

### Importação da biblioteca de limpeza

In [183]:
from modules.utils import data_profiling, remove_special_caracters, process_dataframe, remove_outliers, change_t_for_1

### Relatório com informações do perfil dos dados

In [14]:
# data_profiling(df_silver_listings, 'listings', 'data_profiling/')
# data_profiling(df_silver_reviews, 'reviews', 'data_profiling/')
# data_profiling(df_silver_calendar, 'calendar', 'data_profiling/')

### Processamento da tabela **listings**

##### Limpeza de caracteres

In [184]:
df_silver_listings["neighborhood_overview"] = df_silver_listings["neighborhood_overview"].apply(remove_special_caracters)
df_silver_listings["host_about"] = df_silver_listings["host_about"].apply(remove_special_caracters)

##### Identificar e lidar com valores ausentes, duplicatas e outliers

In [185]:
# colunas para considerar a eliminação de outliers
cols_listings = ['review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

df_process = process_dataframe(df_silver_listings, 'listings')
df_silver_listings = remove_outliers(df_process, cols_listings)

Analisando a tabela listings

Removendo colunas que possuem 100% de valores faltantes ...
Colunas removidas da tabela  listings : 
 ['bathrooms', 'bedrooms', 'calendar_updated', 'description', 'neighbourhood_group_cleansed', 'license']

Removendo dados duplicados ...
Foram removidas  0 linhas da tabela listings

Removendo colunas constantes ...
Colunas constantes removidas da tabela listings: 
 ['scrape_id', 'amenities']

Quantidade de linhas (outliers) eliminadas:  27265


##### Conversão do T e F para 1 e 0

In [186]:
cols_listings2 = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']

for col in cols_listings2:
    df_silver_listings[col] = df_silver_listings[col].apply(change_t_for_1).astype("int")

##### Transformação da coluna bathroom_text em 2 colunas

In [187]:
df_silver_listings['bathrooms_text'].unique()

array(['1 bath', '1.5 baths', '5 baths', '2 baths', '7 baths', '3 baths',
       '1 shared bath', '4 baths', '2.5 baths', '5 shared baths',
       '1 private bath', '2.5 shared baths', '2 shared baths',
       '4.5 baths', '0 shared baths', 'Shared half-bath',
       '1.5 shared baths', '0 baths', None, '3.5 baths', '6 baths',
       '3 shared baths', '5.5 baths', '7.5 baths', '3.5 shared baths',
       '4.5 shared baths', '6.5 shared baths', '6.5 baths', '10 baths',
       '4 shared baths', '16 baths', '8 baths', 'Half-bath'], dtype=object)

In [189]:
# Preencha os valores nulos na coluna 'bathrooms_text' com '0 bath'
df_silver_listings['bathrooms_text'] = df_silver_listings['bathrooms_text'].fillna('0 bath')
    
# Defina o tipo de banheiro
df_silver_listings['bathroom_type'] = df_silver_listings['bathrooms_text'].str.replace(r'(\d+\.?\d*)', '', regex=True)
    
# Remova 's' do final de 'baths e traz o strip()'
df_silver_listings['bathroom_type'] = df_silver_listings['bathroom_type'].str.replace(r's?$', '', regex=True).apply(lambda x: x.strip())
    
# Extraia o número de banheiros usando expressões regulares
df_silver_listings['bathroom_quantity'] = df_silver_listings['bathrooms_text'].str.extract(r'(\d+\.?\d*)')[0].astype(float)
    
# Converta a coluna 'bathroom_quantity' para tipo numérico
df_silver_listings['bathroom_quantity'] = pd.to_numeric(df_silver_listings['bathroom_quantity'], errors='coerce').fillna(0)

In [190]:
df_silver_listings['bathroom_type'].unique()

array(['bath', 'shared bath', 'private bath', 'Shared half-bath',
       'Half-bath'], dtype=object)

In [191]:
df_silver_listings[["bathroom_type",'bathroom_quantity']]

,bathroom_type,bathroom_quantity
0,bath,1.0
1,bath,1.0
2,bath,1.5
3,bath,1.0
4,bath,1.0
...,...,...
32278,bath,1.0
32498,bath,1.0
32739,bath,2.0
33052,bath,1.0


### Processamento da tabela **reviews**

##### Limpeza de caracteres

In [22]:
df_silver_reviews["comments"] = df_silver_reviews["comments"].apply(remove_special_caracters)

##### Identificar e lidar com valores ausentes, duplicatas e outliers

In [23]:
# colunas para considerar a eliminação de outliers
# cols_reviews = []

df_process = process_dataframe(df_silver_reviews, 'reviews')
# df_silver_reviews = remove_outliers(df_process, cols_reviews)

Analisando a tabela reviews

Removendo colunas que possuem 100% de valores faltantes ...
Colunas removidas da tabela  reviews : 
 []

Removendo dados duplicados ...
Foram removidas  0 linhas da tabela reviews

Removendo colunas constantes ...
Colunas constantes removidas da tabela reviews: 
 []


### Processamento da tabela **calendar**

##### Limpeza de caracteres

In [24]:
df_silver_calendar["price"] = df_silver_calendar["price"].apply(remove_special_caracters).astype("float")

##### Identificar e lidar com valores ausentes, duplicatas e outliers

In [25]:
# colunas para considerar a eliminação de outliers
cols_calendar = ['price']

df_process = process_dataframe(df_silver_calendar, 'calendar')
df_silver_calendar = remove_outliers(df_process, cols_calendar)

Analisando a tabela calendar

Removendo colunas que possuem 100% de valores faltantes ...
Colunas removidas da tabela  calendar : 
 ['adjusted_price']

Removendo dados duplicados ...
Foram removidas  0 linhas da tabela calendar

Removendo colunas constantes ...
Colunas constantes removidas da tabela calendar: 
 []

Quantidade de linhas (outliers) eliminadas:  551925


##### Conversão do T e F para 1 e 0

In [26]:
cols_calendar2 = ['available']

for col in cols_calendar2:
    df_silver_calendar[col] = df_silver_calendar[col].apply(change_t_for_1).astype("int")

C:\Users\luana\AppData\Local\Temp\ipykernel_10172\3237782736.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver_calendar[col] = df_silver_calendar[col].apply(change_t_for_1).astype("int")


# Etapa 3:

**Data Quality - Camada Silver**

   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

### Importação da funções de verificação

In [241]:
from modules.utils import check_missing, check_date

ImportError: cannot import name 'check_missing' from 'modules.utils' (c:\repos_GitHub\ada_enginnering_analytics\modules\utils.py)

### Processamento da tabela **listings**

##### Preenchimento dos campos nulos com valores fixos

In [194]:
## Colunas com dados faltantes 
check_missing(df_silver_listings).sort_values(ascending=False)

host_about                   3553
neighborhood_overview        3001
neighbourhood                3001
host_neighbourhood           2085
host_location                1128
host_response_time            508
host_response_rate            508
host_acceptance_rate          344
price                         196
has_availability              196
beds                           65
host_name                       2
host_listings_count             2
host_total_listings_count       2
host_verifications              2
host_thumbnail_url              2
host_since                      2
host_picture_url                2
dtype: int64

In [195]:
# Preenchimento dos campos nulos com "no_info"

cols_no_info = ['neighborhood_overview', 'host_name', 'host_location', 'host_about', 'host_response_time', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications']

for column in cols_no_info:
    df_silver_listings[column] = df_silver_listings[column].fillna("no_info")

In [196]:
# Preenchimento dos campos nulos com outros valores

# Coluna 'neighbourhood' preencher com 'Rio de Janeiro'
df_silver_listings['neighbourhood'] = df_silver_listings['neighbourhood'].fillna("Rio de Janeiro")


# Coluna 'beds' preencher com valor 1 (mínimo necessário para hospedar) 
df_silver_listings['beds'] = df_silver_listings['beds'].fillna(1)


# Coluna 'host_listings_count' e 'host_total_listings_count' preencher com valor 1 (the number of listings the host has)
for col in ['host_listings_count','host_total_listings_count']:
    df_silver_listings[col] = df_silver_listings[col].fillna(1)

# Coluna 'has_availability' preencher com valor 't'
df_silver_listings['has_availability'] = df_silver_listings['has_availability'].fillna('t')

In [197]:
# Coluna 'host_since' preencher com a data atual
data_atual = pd.Timestamp.now().strftime('%Y-%m-%d')

df_silver_listings['host_since'] = df_silver_listings['host_since'].fillna(data_atual)

In [198]:
# Coluna 'price' preencher com a média de preço para hospedagens com quant de camas e banheiros semelhantes

## limpeza de caracteres
df_silver_listings["price"] = df_silver_listings["price"].apply(remove_special_caracters)

## Coverter tipo para float
df_silver_listings['price'] = df_silver_listings['price'].astype("float")

## Calcula a média dos preços agrupando pelos valores das colunas 'beds' e 'bathroom_quantity'
mean_prices = df_silver_listings.groupby(['beds', 'bathroom_quantity'])['price'].mean()

# Preenche os valores nulos na coluna 'price' com as médias calculadas
df_silver_listings['price'] = df_silver_listings.apply(lambda row: mean_prices[row['beds'], row['bathroom_quantity']] if pd.isnull(row['price']) else row['price'], axis=1)


In [199]:
# Dropar coluna 'name'

df_silver_listings.drop('name', axis=1, inplace=True)

In [244]:
# Colunas 'host_response_rate' e 'host_acceptance_rate'  preencher com "0%"

for col in ['host_response_rate', 'host_acceptance_rate']:
    df_silver_listings[col] = df_silver_listings[col].fillna('0%')

In [245]:
## Verificar novamente dados faltantes 
check_missing(df_silver_listings).sort_values(ascending=False)

Series([], dtype: int64)

##### Padronização dos nomes e tipos das colunas

In [201]:
# Definir o tipo das colunas que não serão de texto
list_int_listings = ['id', 'host_id', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
                     'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms',
                     'host_listings_count','host_total_listings_count', 'beds', 'accommodates', 'minimum_nights', 
                     'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 
                     'maximum_maximum_nights', 'availability_30', 'availability_60', 'availability_90', 'availability_365',
                     'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d']


list_float_listings = ['review_scores_location', 'review_scores_value', 'reviews_per_month', 'price', 'minimum_nights_avg_ntm', 
                       'maximum_nights_avg_ntm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
                       'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']


list_datetime_listings = ['last_scraped', 'host_since', 'calendar_last_scraped', 'first_review', 'last_review']


list_bool_listings = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 
                      'has_availability', 'instant_bookable']

In [202]:
## Definir tipos das colunas

for column in list_int_listings:
    # transforma a coluna para o tipo inteiro
    df_silver_listings[column] = df_silver_listings[column].astype("int")

for column in list_float_listings:
    # transforma a coluna para o tipo float
    df_silver_listings[column] = df_silver_listings[column].astype("float")

for column in list_datetime_listings:
    # transforma a coluna para o tipo "datetime" com o formato passado
    df_silver_listings[column] = pd.to_datetime(df_silver_listings[column], format='%Y-%m-%d')

##### Conversão do T e F para 1 e 0

In [203]:
df_silver_listings['has_availability'] = df_silver_listings['has_availability'].apply(change_t_for_1).astype("int")

In [204]:
df_silver_listings[list_bool_listings]

,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable
0,1,1,1,1,0
1,1,1,1,1,0
2,1,1,1,1,0
3,0,1,1,1,0
4,0,1,0,1,0
...,...,...,...,...,...
32278,0,1,1,1,1
32498,0,1,1,1,0
32739,0,1,0,1,0
33052,0,1,1,1,0


##### Verificação da precisão dos dados

In [231]:
## Verificação de datas 

for col in list_datetime_listings:
    df_filtrado, valores_fora_intervalo = check_date(df_silver_listings, col)

    print('Analisando datas da coluna ' + col)

    if not valores_fora_intervalo.notnull().sum().any():
        print('  - Nenhum valor de data fora do intervalo\n')
    else:
        print('  - Existem bad datas na coluna\n')

Analisando datas da coluna last_scraped
  - Nenhum valor de data fora do intervalo

Analisando datas da coluna host_since
  - Nenhum valor de data fora do intervalo

Analisando datas da coluna calendar_last_scraped
  - Nenhum valor de data fora do intervalo

Analisando datas da coluna first_review
  - Nenhum valor de data fora do intervalo

Analisando datas da coluna last_review
  - Nenhum valor de data fora do intervalo



In [232]:
## Verificação de preços  

df_silver_listings[df_silver_listings['price']<=0]

,id,listing_url,last_scraped,source,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,bathroom_type,bathroom_quantity


### Processamento da tabela **reviews**

##### Preenchimento dos campos nulos com valores fixos

In [30]:
## Colunas com dados faltantes + quant de dados faltantes
check_missing(df_silver_reviews).sort_values(ascending=False)

comments    20
dtype: int64

In [34]:
df_silver_reviews['comments'] = df_silver_reviews['comments'].fillna("no_info")

In [35]:
## Verificar novamente dados faltantes 
check_missing(df_silver_reviews).sort_values(ascending=False)

Series([], dtype: int64)

##### Padronização dos nomes e tipos das colunas

In [36]:
# Definir o tipo das colunas que não serão de texto
list_int_reviews = ['listing_id', 'id', 'reviewer_id']
list_datetime_reviews = ['date']

In [37]:
df_silver_reviews.dtypes

listing_id        int64
id                int64
date             object
reviewer_id       int64
reviewer_name    object
comments         object
dtype: object

In [38]:
## Apenas a alteração do tipo da coluna 'date' é necessária.
df_silver_reviews['date'] = pd.to_datetime(df_silver_reviews['date'], format='%Y-%m-%d')

In [39]:
df_silver_reviews.dtypes

listing_id                int64
id                        int64
date             datetime64[ns]
reviewer_id               int64
reviewer_name            object
comments                 object
dtype: object

##### Verificação da precisão dos dados

In [208]:
## Verificação de datas 

df_filtrado, valores_fora_intervalo = check_date(df_silver_reviews, 'date')

In [215]:
## Nenhum valor de data fora do intervalo 
valores_fora_intervalo

,listing_id,id,date,reviewer_id,reviewer_name,comments


### Processamento da tabela **calendar**

##### Preenchimento dos campos nulos com o valor que mais se repete

In [42]:
## Colunas com dados faltantes + quant de dados faltantes
check_missing(df_silver_calendar).sort_values(ascending=False)

minimum_nights    23
maximum_nights    23
dtype: int64

In [43]:
minimum_freq = df_silver_calendar['minimum_nights'].value_counts().idxmax()
maximum_freq = df_silver_calendar['maximum_nights'].value_counts().idxmax()

In [44]:
df_silver_calendar['minimum_nights'] = df_silver_calendar['minimum_nights'].fillna(minimum_freq)
df_silver_calendar['maximum_nights'] = df_silver_calendar['maximum_nights'].fillna(maximum_freq)

C:\Users\luana\AppData\Local\Temp\ipykernel_10172\1914255510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver_calendar['minimum_nights'] = df_silver_calendar['minimum_nights'].fillna(minimum_freq)
C:\Users\luana\AppData\Local\Temp\ipykernel_10172\1914255510.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver_calendar['maximum_nights'] = df_silver_calendar['maximum_nights'].fillna(maximum_freq)


In [45]:
## Verificar novamente dados faltantes 
check_missing(df_silver_calendar).sort_values(ascending=False)

Series([], dtype: int64)

##### Padronização dos nomes e tipos das colunas

In [46]:
# Definir o tipo das colunas que não serão de texto
list_int_calendar = ['listing_id', 'minimum_nights', 'maximum_nights']
list_datetime_calendar = ['date']
list_float_calendar = ['price']

In [47]:
df_silver_calendar.dtypes

listing_id          int64
date               object
available           int32
price             float64
minimum_nights    float64
maximum_nights    float64
dtype: object

In [48]:
## Nessário apenas alterar o tipo das colunas 'date','minimum_nights', 'maximum_nights'

df_silver_calendar['date'] = pd.to_datetime(df_silver_calendar['date'], format='%Y-%m-%d')

for column in ['minimum_nights', 'maximum_nights']:
    #transforma a coluna para o tipo inteiro
    df_silver_calendar[column] = df_silver_calendar[column].astype("int")

C:\Users\luana\AppData\Local\Temp\ipykernel_10172\2483845584.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver_calendar['date'] = pd.to_datetime(df_silver_calendar['date'], format='%Y-%m-%d')
C:\Users\luana\AppData\Local\Temp\ipykernel_10172\2483845584.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver_calendar[column] = df_silver_calendar[column].astype("int")
C:\Users\luana\AppData\Local\Temp\ipykernel_10172\2483845584.py:7: SettingWithCopyWarning: 
A value is trying to be set on a 

In [49]:
## Verificar novamente tipos das colunas
df_silver_calendar.dtypes

listing_id                 int64
date              datetime64[ns]
available                  int32
price                    float64
minimum_nights             int32
maximum_nights             int32
dtype: object

##### Verificação da precisão dos dados

In [217]:
## Verificação de datas 

df_filtrado, valores_fora_intervalo = check_date(df_silver_calendar, 'date')

In [218]:
## Nenhum valor de data fora do intervalo 
valores_fora_intervalo

,listing_id,date,available,price,minimum_nights,maximum_nights


In [53]:
## Verificação de preços  

df_silver_calendar[df_silver_calendar['price']<=0]

,listing_id,date,available,price,minimum_nights,maximum_nights


# Etapa 4:

**Testes de Qualidade - Camada Silver**

   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

# Etapa 5:

**Transformação de Dados com dbt - Camada Silver**

   - Utilize a ferramenta dbt para criar a camada "silver" de dados, realizando transformações e preparando os dados da camada em questão.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "silver" e automatize a execução das transformações.

# Etapa 6:

**Armazenamento de Dados em PostgreSQL - Camada Silver**

   - Armazene os dados da camada "silver" no mesmo banco de dados PostgreSQL.
   - Estabeleça conexões entre o dbt e o PostgreSQL para carregar os dados transformados da camada "silver" no banco.

# Etapa 7:

**Validação de Expectativas com Great Expectations - Camada Silver**

   - Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
   - Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

# Etapa 8:

**Transformação de Dados com dbt - Camada Gold**

   - Utilize o dbt para criar a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "gold" e automatize a execução das transformações.
   - Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.